# Accessibility of info online

In [35]:
import pandas as pd
from functools import reduce #necessary to create the last dataframe
df = pd.read_csv(r"C:\Users\hp pc\Desktop\Tesi\Smart cities\Databases\2. Citizen Centrality\Transparency\urb_percep_1.csv", encoding = "ISO-8859-1",index_col=0)
del df["UNIT"] #Delete the unit of measurement
# Rename cities in English
df = df.replace(["Athina (greater city)"],"Athens")
df = df.replace(['Bruxelles / Brussel'],'Bruxelles')
df = df.replace(['København (greater city)'],'Copenhagen')
df = df.replace(['Helsinki / Helsingfors (greater city)'],'Helsinki')
df = df.replace(['Madrid'],'Madrid')
df = df.replace(['Paris (greater city)'],'Paris')
df = df.replace(['Praha'],'Prague')
df = df.replace(['Roma'],'Rome')
df = df.replace(['Stockholm (greater city)'],'Stockholm')
df = df.replace(['Warszawa'],'Warsaw')
df = df.replace(['Wien'],'Vienna')
df = df.replace(['Zagreb'],'Zagreb')

#these are the cities that we analyse in our study
cities=["Athens","Bruxelles","Copenhagen", "Helsinki", "Madrid", "Paris", "Prague", "Rome", "Stockholm",
        "Warsaw","Vienna", "Zagreb"]
df = df.rename(columns={"GEO": "CITIES"}) # RENAME COLUMNS
#df = df.rename(columns={"Value": "Information and services of my local public administration can be easily accessed online: strongly agree"}) 
df.sort_values("CITIES", axis=0, ascending=True, inplace=True)
# make list containing names of the variables (no duplicates)
indicators = list(dict.fromkeys(df["INDIC_UR"])) 
dfList=[] #create an empty list that will filled with dataframes
for nome_indicatore in indicators:
    df1=df[df.INDIC_UR == nome_indicatore] 
    for city in cities: 
        sub_df=df1[df1.CITIES == city]
        dfList.append(sub_df)
df=(pd.concat(dfList,ignore_index=False)) # This df containes all variables: it needs to be splitted...

final_dfs=[]     #...split the previous df and creating the complete one
for indicator in indicators:
    df1=df[df.INDIC_UR == indicator] 
    df1 = df1.rename(columns={"Value": indicator}) #rename "Value" with the name of the variable 
    del df1["INDIC_UR"] # delete the column filled with the name of the variable. 
    df1.reset_index(drop=True, inplace=True) 
    df1.sort_values("CITIES", axis=0, ascending=True, inplace=True)
    final_dfs.append(df1) # this is the list of df that need to be merged
df= reduce(lambda x, y: pd.merge(x, y, on = 'CITIES'), final_dfs) #merge dfs on CITIES one next to the other
df

,CITIES,Information and services of my local public administration can be easily accessed online: strongly agree,Information and services of my local public administration can be easily accessed online: somewhat agree
0,Athens,22.4,30.2
1,Bruxelles,41.0,29.1
2,Copenhagen,41.2,42.1
3,Helsinki,39.3,33.0
4,Madrid,38.7,24.4
5,Paris,44.8,26.1
6,Prague,32.2,39.5
7,Rome,6.3,45.9
8,Stockholm,29.0,38.5
9,Vienna,39.8,42.1


In [36]:
# RENAME COLUMNS USING EASIER NAMES and 
df.columns=["CITIES","info_acc_str","info_acc_som"]
# CREATE NEW COLUMNS AS SUM OF VARIABLES "STRONG AGREE" AND "SOMEWHAT AGREE"
df["info_acc"]=df["info_acc_str"] + df["info_acc_som"] 
# DELETE USELESS COLUMNS
del df["info_acc_str"]
del df["info_acc_som"] 

# Apply min-max normalization formula to each value in df 
df["info_acc"]=[round(((s-min(df.info_acc))/(max(df.info_acc)-min(df.info_acc)))*10,1) for s in df["info_acc"]]
df1=df
df1

,CITIES,info_acc
0,Athens,0.1
1,Bruxelles,5.8
2,Copenhagen,10.0
3,Helsinki,6.5
4,Madrid,3.5
5,Paris,6.0
6,Prague,6.3
7,Rome,0.0
8,Stockholm,4.9
9,Vienna,9.5


# PERSONAL EXPERIMENT MUNICIPALITY REPLIES

In [37]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
df = pd.read_excel(r"C:\Users\hp pc\Desktop\Tesi\Smart cities\Databases\2. Citizen Centrality\Transparency\Municipality Replies.xlsx", sheetname='Foglio1') 
df

C:\Users\hp pc\Documents\Python-Anaconda\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


,CITIES,email_score
0,Athens,1.50
1,Bruxelles,6.00
2,Copenhagen,4.50
3,Helsinki,3.25
4,Madrid,0.00
5,Paris,4.50
6,Prague,6.00
7,Rome,4.50
8,Stockholm,6.00
9,Warsaw,3.75


In [38]:
#Apply normalization
df["email_score"]=[round(((s-min(df.email_score))/(max(df.email_score)-min(df.email_score)))*10,2) for s in df["email_score"]]
df2=df
df2

,CITIES,email_score
0,Athens,2.50
1,Bruxelles,10.00
2,Copenhagen,7.50
3,Helsinki,5.42
4,Madrid,0.00
5,Paris,7.50
6,Prague,10.00
7,Rome,7.50
8,Stockholm,10.00
9,Warsaw,6.25


In [39]:
df = pd.merge(df1, df2, on=['CITIES'])
#CREATE LAST COLUMN
df['Transparency_Acc_Index'] = round(df.mean(axis=1),2)
Transparency_Acc_Index=df
Transparency_Acc_Index

,CITIES,info_acc,email_score,Transparency_Acc_Index
0,Athens,0.1,2.50,1.30
1,Bruxelles,5.8,10.00,7.90
2,Copenhagen,10.0,7.50,8.75
3,Helsinki,6.5,5.42,5.96
4,Madrid,3.5,0.00,1.75
5,Paris,6.0,7.50,6.75
6,Prague,6.3,10.00,8.15
7,Rome,0.0,7.50,3.75
8,Stockholm,4.9,10.00,7.45
9,Vienna,9.5,7.50,8.50


In [40]:
Transparency_Acc_Index=Transparency_Acc_Index[["CITIES","Transparency_Acc_Index"]]
Transparency_Acc_Index

,CITIES,Transparency_Acc_Index
0,Athens,1.30
1,Bruxelles,7.90
2,Copenhagen,8.75
3,Helsinki,5.96
4,Madrid,1.75
5,Paris,6.75
6,Prague,8.15
7,Rome,3.75
8,Stockholm,7.45
9,Vienna,8.50


In [41]:
%store Transparency_Acc_Index

Stored 'Transparency_Acc_Index' (DataFrame)
